In [ ]:
# | default_exp solving

In [ ]:
# | include: false

from nbdev.showdoc import show_doc

In [ ]:
# | export
from itertools import chain

import numpy as np
import pandas as pd
from cobra import flux_analysis
from cobra.core.solution import get_solution
from cobra.util import solver as sutil
from mmon_gcm.buildingediting import check_number_of_models
from optlang.symbolics import Zero

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | export


def pfba_Weighted(
    model, weightings=None, fraction_of_optimum=1.0, objective=None, reactions=None
):
    """Perform basic pFBA (parsimonious Enzyme Usage Flux Balance Analysis)
    to minimize total flux.
    pFBA [1] adds the minimization of all fluxes the the objective of the
    model. This approach is motivated by the idea that high fluxes have a
    higher enzyme turn-over and that since producing enzymes is costly,
    the cell will try to minimize overall flux while still maximizing the
    original objective function, e.g. the growth rate.

    ##### Parameters:
    model : cobra.Model
        The model
    fraction_of_optimum : float, optional
        Fraction of optimum which must be maintained. The original objective
        reaction is constrained to be greater than maximal_value *
        fraction_of_optimum.
    objective : dict or model.problem.Objective
        A desired objective to use during optimization in addition to the
        pFBA objective. Dictionaries (reaction as key, coefficient as value)
        can be used for linear objectives.
    reactions : iterable
        List of reactions or reaction identifiers. Implies `return_frame` to
        be true. Only return fluxes for the given reactions. Faster than
        fetching all fluxes if only a few are needed.

    ##### Returns:
    cobra.Solution

    The solution object to the optimized model with pFBA constraints added.

    References:
    .. [1] Lewis, N. E., Hixson, K. K., Conrad, T. M., Lerman, J. A.,
       Charusanti, P., Polpitiya, A. D., Palsson, B. O. (2010). Omic data
       from evolved E. coli are consistent with computed optimal growth from
       genome-scale models. Molecular Systems Biology, 6,
       390. doi:10.1038/msb.2010.47
    """
    reactions = (
        model.reactions if reactions is None else model.reactions.get_by_any(reactions)
    )
    tempmodel = model.copy()
    with tempmodel as m:
        add_pfba_Weighted(
            m, weightings, objective=objective, fraction_of_optimum=fraction_of_optimum
        )
        m.slim_optimize(error_value=None)
        solution = get_solution(m, reactions=reactions)
    return m, solution

In [ ]:
show_doc(pfba_Weighted)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L19){target="_blank" style="float:right; font-size:smaller"}

### pfba_Weighted

>      pfba_Weighted (model, weightings=None, fraction_of_optimum=1.0,
>                     objective=None, reactions=None)

Perform basic pFBA (parsimonious Enzyme Usage Flux Balance Analysis)
to minimize total flux.
pFBA [1] adds the minimization of all fluxes the the objective of the
model. This approach is motivated by the idea that high fluxes have a
higher enzyme turn-over and that since producing enzymes is costly,
the cell will try to minimize overall flux while still maximizing the
original objective function, e.g. the growth rate.

##### Parameters:
model : cobra.Model
    The model
fraction_of_optimum : float, optional
    Fraction of optimum which must be maintained. The original objective
    reaction is constrained to be greater than maximal_value *
    fraction_of_optimum.
objective : dict or model.problem.Objective
    A desired objective to use during optimization in addition to the
    pFBA objective. Dictionaries (reaction as key, coefficient as value)
    can be used for linear objectives.
reactions : iterable
    List of reactions or reaction identifiers. Implies `return_frame` to
    be true. Only return fluxes for the given reactions. Faster than
    fetching all fluxes if only a few are needed.

##### Returns:
cobra.Solution

The solution object to the optimized model with pFBA constraints added.

References:
.. [1] Lewis, N. E., Hixson, K. K., Conrad, T. M., Lerman, J. A.,
   Charusanti, P., Polpitiya, A. D., Palsson, B. O. (2010). Omic data
   from evolved E. coli are consistent with computed optimal growth from
   genome-scale models. Molecular Systems Biology, 6,
   390. doi:10.1038/msb.2010.47

In [ ]:
# | export


def add_pfba_Weighted(model, weightings=None, objective=None, fraction_of_optimum=1.0):
    """
    This function is a modified version of cobrapy add_pfba function

    Add pFBA objective
    Add objective to minimize the summed flux of all reactions to the
    current objective.

    See Also:
    pfba

    Parameters:
    model : cobra.Model
        The model to add the objective to
    objective :
        An objective to set in combination with the pFBA objective.
    fraction_of_optimum : float
        Fraction of optimum which must be maintained. The original objective
        reaction is constrained to be greater than maximal_value *
        fraction_of_optimum.
    """
    if weightings == None:
        weightings = get_weightings(model)
    if objective is not None:
        model.objective = objective
    if model.solver.objective.name == "_pfba_objective":
        raise ValueError("The model already has a pFBA objective.")
    sutil.fix_objective_as_constraint(model, fraction=fraction_of_optimum)
    reaction_variables = (
        (rxn.forward_variable, rxn.reverse_variable) for rxn in model.reactions
    )
    variables = chain(*reaction_variables)
    model.objective = model.problem.Objective(
        Zero, direction="min", sloppy=True, name="_pfba_objective"
    )
    # print([v for v in variables])
    tempDict = dict()
    for v in variables:
        w = str(v).split("=")[1].replace(" ", "").replace("<", "")
        found = False
        for rxn in weightings.keys():
            if w.__contains__(rxn):
                tempDict[v] = weightings[rxn]
                found = True
                break
        if not found:
            print(
                "Weightings for reaction " + w + " not found, so assuming weighting = 1"
            )
            tempDict[v] = 1
    model.objective.set_linear_coefficients(tempDict)

In [ ]:
show_doc(add_pfba_Weighted)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L71){target="_blank" style="float:right; font-size:smaller"}

### add_pfba_Weighted

>      add_pfba_Weighted (model, weightings=None, objective=None,
>                         fraction_of_optimum=1.0)

This function is a modified version of cobrapy add_pfba function

Add pFBA objective
Add objective to minimize the summed flux of all reactions to the
current objective.

See Also:
pfba

Parameters:
model : cobra.Model
    The model to add the objective to
objective :
    An objective to set in combination with the pFBA objective.
fraction_of_optimum : float
    Fraction of optimum which must be maintained. The original objective
    reaction is constrained to be greater than maximal_value *
    fraction_of_optimum.

In [ ]:
# | export


def get_weightings(model):
    """
    This function is used by pfba_weighted to generate default weightings for the guard cell model
    It takes the model as an argument and returns the weightings based on the phase lengths of the model.
    """
    weightings = {}
    number_of_models = check_number_of_models(model)
    for i in range(1, number_of_models + 1):
        length_of_phase = 1 / (
            -model.reactions.get_by_id(f"SUCROSE_v_gc_Linker_{i}").get_coefficient(
                f"SUCROSE_v_gc_{i}"
            )
        )
        for reaction in model.reactions:
            if (
                "constraint" in reaction.id
                or "overall" in reaction.id
                or "sum" in reaction.id
                or reaction.id[:2] == "EX"
            ):
                weightings[reaction.id] = 0
            elif reaction.id[-1] == str(i):
                weightings[reaction.id] = length_of_phase
    return weightings

In [ ]:
show_doc(get_weightings)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L124){target="_blank" style="float:right; font-size:smaller"}

### get_weightings

>      get_weightings (model)

This function is used by pfba_weighted to generate default weightings for the guard cell model
It takes the model as an argument and returns the weightings based on the phase lengths of the model.

In [ ]:
# | export


def pFBA_FVA_run(cobra_model, obj, rxnlist=[], processes=3, fix_sof_for_fva=False):

    print("Running pFBA")
    cobra_model, solution = pfba_Weighted(cobra_model, objective=obj)
    # pfba_model = cobra_model.copy()
    objvalue = solution.get_primal_by_id(obj)

    if fix_sof_for_fva == True:

        sum_of_fluxes = get_sum_of_fluxes(cobra_model)

        # get the weightings for SOF and generate a copy of the model that, where weightings are not zero and the reaction is reversible,
        # splits the reaction into a forwards and reverse reaction and makes them both irreversible
        weightings = get_weightings(cobra_model)
        cobra_model2 = cobra_model.copy()
        irr_model = rev2irrev(cobra_model2)
        print("Setting SOF model")

        # set the weightings for the reverse reactions to be the same as the forward reactions
        for reaction in irr_model.reactions:
            if reaction.id.__contains__("_reverse"):
                id = reaction.id
                originalreaction = id.replace("_reverse", "")
                weightings[reaction.id] = weightings[originalreaction]

        # weight the forward and reverse reactions of the reactions, whether they are forward or reverse, equally for SOF.
        # Add a constraint to the model that the sum of these reactions with their coefficients cannot be different to the sum_of_fluxes from pFBA
        coefficients = {}
        for reaction in irr_model.reactions:
            coefficients[reaction.forward_variable] = weightings[reaction.id]
            coefficients[reaction.reverse_variable] = weightings[reaction.id]
        sofconstraint = irr_model.problem.Constraint(
            0, lb=sum_of_fluxes, ub=sum_of_fluxes, name="sofconstraint"
        )
        irr_model.add_cons_vars(sofconstraint)
        irr_model.solver.update()
        sofconstraint.set_linear_coefficients(coefficients=coefficients)

        new_coefficients = coefficients.copy()
        sum_of_fluxes = irr_model.problem.Variable("sum_of_fluxes")
        new_coefficients[sum_of_fluxes] = -1
        sofvariableconstraint = irr_model.problem.Constraint(0, lb=0, ub=0)
        irr_model.add_cons_vars([sum_of_fluxes, sofvariableconstraint])
        irr_model.solver.update()
        sofvariableconstraint.set_linear_coefficients(coefficients=new_coefficients)

        # fix objective to be equal to pFBA
        phloemconstraint = irr_model.problem.Constraint(
            irr_model.reactions.get_by_id(obj).flux_expression,
            lb=objvalue,
            ub=objvalue,
            name="phloem_output",
        )
        irr_model.add_cons_vars(phloemconstraint)

        irr_model.optimize()

        sfmodel = irr_model.copy()

        rxnlist2 = []

        # if rxnlist is not empty, add just the forward reaction if it isn't reversible or add both forward and reverse if it is
        for rxn in rxnlist:
            rxn = sfmodel.reactions.get_by_id(rxn.id)
            if rxn.lower_bound < 0 and rxn.upper_bound > 0 and weightings[rxn.id] != 0:
                rxnlist2.append(sfmodel.reactions.get_by_id(rxn.id + "_reverse"))
            rxnlist2.append(sfmodel.reactions.get_by_id(rxn.id))

        print("Running FVA")

        fva = flux_analysis.flux_variability_analysis(sfmodel, reaction_list=rxnlist2, processes=processes)
        print("Processing results")

        fva2 = dict()
        for mode in fva.keys():
            if mode == "maximum":
                tempdict = dict()
                FVArxnSet = set()
                for rxn in fva[mode].keys():
                    if rxn.__contains__("_reverse"):
                        rxn = rxn.replace("_reverse", "")
                    if FVArxnSet.__contains__(rxn):
                        continue
                    FVArxnSet.add(rxn)
                    if not fva[mode].keys().__contains__(rxn + "_reverse"):
                        maxi = fva[mode][rxn]
                    else:
                        maxi = fva[mode][rxn] + fva[mode][rxn + "_reverse"]
                    tempdict[rxn] = maxi
            else:
                tempdict = dict()
                FVArxnSet = set()
                for rxn in fva[mode].keys():
                    if rxn.__contains__("_reverse"):
                        rxn = rxn.replace("_reverse", "")
                    if FVArxnSet.__contains__(rxn):
                        continue
                    FVArxnSet.add(rxn)
                    if not fva[mode].keys().__contains__(rxn + "_reverse"):
                        mini = fva[mode][rxn]
                    else:
                        mini = fva[mode][rxn] + fva[mode][rxn + "_reverse"]
                    tempdict[rxn] = mini
            fva2[mode] = tempdict

        cobra_model.fva = fva2

    else:
        if len(rxnlist) == 0:
            print("FVA list is empty")
        else:
            print(f"Running FVA on {len(rxnlist)} reactions using {processes} processes")

        fva = flux_analysis.flux_variability_analysis(
            cobra_model, reaction_list=rxnlist, processes=processes, loopless=True)
        cobra_model.fva = fva

    return cobra_model, solution

In [ ]:
show_doc(pFBA_FVA_run)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L150){target="_blank" style="float:right; font-size:smaller"}

### pFBA_FVA_run

>      pFBA_FVA_run (cobra_model, obj, rxnlist=[], processes=3,
>                    fix_sof_for_fva=False)

In [ ]:
# | export

def get_sum_of_fluxes(model):
    weightings = get_weightings(model)
    sum_of_fluxes = 0
    for reaction_id in weightings.keys():
        sum_of_fluxes = sum_of_fluxes + (
            abs(model.reactions.get_by_id(reaction_id).flux) * weightings[reaction_id]
        )
    return sum_of_fluxes

In [ ]:
show_doc(get_sum_of_fluxes)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L270){target="_blank" style="float:right; font-size:smaller"}

### get_sum_of_fluxes

>      get_sum_of_fluxes (model)

In [ ]:
# | export

def rev2irrev(cobra_model):
    """
    Function to convert any model with reversible reactions to a copy of the same m-
    -odel with only irreversible reactions. ID of reverse reactions are generated by
    suffixing "_reverse" to the ID of the orignal reaction.
    args: 1) a cobra model
    output: a cobra model with only irreversible reactions
    """
    exp_model = cobra_model.copy()
    for RXN in cobra_model.reactions:
        rxn = exp_model.reactions.get_by_id(RXN.id)
        if rxn.lower_bound < 0:
            rxn_reverse = rxn.copy()
            rxn_reverse.id = "%s_reverse" % (rxn.id)
            rxn.lower_bound = 0
            rxn_reverse.upper_bound = 0
            exp_model.add_reaction(rxn_reverse)

    return exp_model

In [ ]:
show_doc(rev2irrev)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L280){target="_blank" style="float:right; font-size:smaller"}

### rev2irrev

>      rev2irrev (cobra_model)

Function to convert any model with reversible reactions to a copy of the same m-
-odel with only irreversible reactions. ID of reverse reactions are generated by
suffixing "_reverse" to the ID of the orignal reaction.
args: 1) a cobra model
output: a cobra model with only irreversible reactions

In [ ]:
# | export


def check_fba_fva_run(fba_model, pfba_solution):
    '''
    This is a test that checks if the fluxes that are returned 
    by the fba model are different to those by the pFBA
    '''
    fva_fluxes = np.array(
        [reaction.flux for reaction in fba_model.reactions]
    )
    if len(pfba_solution[pfba_solution.fluxes - fva_fluxes != 0]) > 0:
        return False
    else:
        return True

In [ ]:
show_doc(check_fba_fva_run)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L301){target="_blank" style="float:right; font-size:smaller"}

### check_fba_fva_run

>      check_fba_fva_run (fba_model, pfba_solution)

This is a test that checks if the fluxes that are returned 
by the fba model are different to those by the pFBA

In [ ]:
# | export

def get_pfba_fva_solution(
    fba_model, rxn_list=[], objective="Phloem_tx_overall", processes=3, fix_sof_for_fva=False
):
    """Take FBA model and solve weighted FVA on a list of reactions. 
    Returns a dataframe with flux of all reactions and minimum and 
    maximum if they were in the fva list"""
    fba_model, pfba_solution = pFBA_FVA_run(
        fba_model, objective, rxnlist=rxn_list, processes=processes, fix_sof_for_fva=fix_sof_for_fva
    )

    pfba_df = pfba_solution.to_frame().loc[:, "fluxes":"fluxes"]
    fva_df = pd.DataFrame(fba_model.fva)

    combined_df = pfba_df.join(fva_df)

    #assert(check_fba_fva_run(fba_model, pfba_solution))

    return fba_model, combined_df

In [ ]:
show_doc(get_pfba_fva_solution)

---

[source](https://github.com/noahsprent/mmon_gcm/tree/main/blob/main/mmon_gcm/solving.py#L315){target="_blank" style="float:right; font-size:smaller"}

### get_pfba_fva_solution

>      get_pfba_fva_solution (fba_model, rxn_list=[],
>                             objective='Phloem_tx_overall', processes=3,
>                             fix_sof_for_fva=False)

Take FBA model and solve weighted FVA on a list of reactions. 
Returns a dataframe with flux of all reactions and minimum and 
maximum if they were in the fva list